# Wrangle Data Together

Script assumes 8 files:
- ORD weather data
- ORD taxi data
- ORD flight data
- ORD seats data
- Airline fleet matrices
- Tail number data
- FAA aircraft indexes
- Plane names matching file

Weather Variables:
https://mesonet.agron.iastate.edu/request/download.phtml?network=IL_ASOS

Flight Variables:
https://www.transtats.bts.gov/Fields.asp?Table_ID=236

## Load Packages

In [232]:
import pandas as pd
import numpy as np
import re

## Load data

In [292]:
ORD_weather = pd.read_csv("../data/ORD_weather.txt", sep =  ",", skiprows = 5)
ORD_outbound = pd.read_csv("../data/ORD_outbound.csv")
ORD_OTP = pd.read_csv("../data/ORD_OTP.csv")
ORD_seats = pd.read_csv('../data/ORD_seats.csv')
seat_counts_wiki = pd.read_csv("../data/seat_counts_wiki.csv")
plane_details = pd.read_csv("../data/plane_details.csv", usecols=['tail_num', 'manufacturer', 'model'])
aircraft_types = pd.read_csv("../data/L_AIRCRAFT_TYPE.csv_")
plane_names = pd.read_csv("../data/plane_names.csv")

/home/dkruszew/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preliminary Exploratory Data Analysis

In [75]:
tails_list = plane_details.tail_num.unique().tolist()
print("Percent of ORD_OTP flights with tail number:", round((ORD_OTP.shape[0] - ORD_OTP[~ORD_OTP['TAIL_NUM'].isin(tails_list)].shape[0])/ORD_OTP.shape[0], 3))

Percent of ORD_OTP flights with tail number: 0.74


#### ORD_OTP Airline Distribution:

In [83]:
ORD_OTP['OP_UNIQUE_CARRIER'].value_counts()

UA    483352
AA    413998
OO    361638
MQ    358550
EV    248383
DL     58473
NK     50266
YX     32262
F9     19261
US     18900
B6     17044
AS     15951
YV      9548
VX      8161
9E      7450
OH      2870
Name: OP_UNIQUE_CARRIER, dtype: int64

In [90]:
print("Unique Carriers:", ORD_OTP['OP_UNIQUE_CARRIER'].unique().shape[0])

Unique Carriers: 16


#### Plane Manufacturer Distribution:

In [85]:
pd.DataFrame(plane_details['manufacturer'].value_counts()).head(20)

,manufacturer
BOEING,1560
AIRBUS,765
BOMBARDIER INC,689
EMBRAER,472
AIRBUS INDUSTRIE,409
EMBRAER S A,230
MCDONNELL DOUGLAS,117
MCDONNELL DOUGLAS AIRCRAFT CO,73
CESSNA,36
EMBRAER-EMPRESA BRASILEIRA DE,19


In [91]:
print("Unique Manufacturers:", plane_details['manufacturer'].unique().shape[0])

Unique Manufacturers: 69


Manufacturers in Wikipedia data:

In [96]:
pd.DataFrame(seat_counts_wiki.aircraft.str.split(" ", expand=True)[0].unique())

,0
0,Bombardier
1,Airbus
2,Boeing
3,Embraer
4,McDonnell


#### Plane Model Distribution:

In [86]:
pd.DataFrame(plane_details['model'].value_counts()).head(20)

,model
CL-600-2B19,337
A320-232,318
737-823,278
ERJ 170-200 LR,227
CL-600-2C10,200
A321-231,192
EMB-145LR,164
CL-600-2D24,155
737-824,135
737-924ER,130


In [121]:
print("Unique Models:", plane_details['model'].unique().shape[0])

Unique Models: 202


In [122]:
print("Number of models in FAA:", aircraft_types.shape[0])

Number of models in FAA: 425


## Wrangle weather data

In [97]:
# Remove unnecessary columns
ORD_weather = ORD_weather.drop(columns=['station', 'lon', 'lat', 'drct', 'alti', 'mslp', 'gust', 'skyc4', 'skyl4', 'wxcodes', 'ice_accretion_1hr', 'ice_accretion_3hr', 'ice_accretion_6hr', 'peak_wind_gust', 'peak_wind_drct', 'peak_wind_time', 'metar'])
# Obtain datetime
ORD_weather = ORD_weather.rename(columns={'valid' : 'date'})
ORD_weather['date'] = pd.to_datetime(ORD_weather['date'])
ORD_weather = ORD_weather.set_index("date")
# Change missing values to nan
ORD_weather = ORD_weather.replace('M', np.nan)
# Change columns data types so that averaging can occur
numeric_weather_features = ['tmpf', 'dwpf', 'relh', 'sknt', 'p01i', 'vsby', 'skyl1', 'skyl2', 'skyl3', 'feel']
categorical_weather_features = ['skyc1', 'skyc2', 'skyc3']
ORD_weather[numeric_weather_features] = ORD_weather[numeric_weather_features].apply(pd.to_numeric)
ORD_weather[categorical_weather_features] = ORD_weather[categorical_weather_features].astype('category')

**TO DO: Figure out converting categoricals to ordinal for averaging and inclusion.**

In [98]:
# Strip out time categories
ORD_weather['year'] = ORD_weather.index.year
ORD_weather['month'] = ORD_weather.index.month
ORD_weather['day'] = ORD_weather.index.day
ORD_weather['hour'] = ORD_weather.index.hour
# Average columns by hour
ORD_weather_hourly = ORD_weather.groupby(['year', 'month', 'day', 'hour']).mean().reset_index()

## Wrangle Taxi Data

In [299]:
# Remove unnecessary columns
ORD_outbound = ORD_outbound.drop(columns=['Unnamed: 0', 'pickup_community_area'])

## Wrangle Flight Data

In [100]:
ORD_OTP = ORD_OTP.drop(columns=['Unnamed: 0'])
ORD_OTP['FL_DATE'] = pd.to_datetime(ORD_OTP['FL_DATE'])

In [61]:
# ORD_OTP['year'] = pd.DatetimeIndex(ORD_OTP['FL_DATE']).year
# ORD_OTP['month'] = pd.DatetimeIndex(ORD_OTP['FL_DATE']).month
# ORD_OTP['day'] = pd.DatetimeIndex(ORD_OTP['FL_DATE']).day
# ORD_OTP = ORD_OTP.rename(columns={'ARR_HOUR':'hour'})
# ORD_OTP = ORD_OTP.dropna()
# ORD_OTP.astype({'hour': 'int64'})
# ORD_OTP = ORD_OTP.drop(columns=["FL_DATE"])

In [101]:
ORD_OTP

,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,ARR_TIME,ARR_DELAY,ARR_HOUR
0,1,2013-01-07,AA,N3DEAA,618,SFO,13930,1393002,ORD,7.0,-8.0,0.0
1,1,2013-01-14,AA,N3DDAA,618,SFO,13930,1393002,ORD,10.0,-5.0,0.0
2,1,2013-01-21,AA,N3DJAA,618,SFO,13930,1393002,ORD,7.0,-8.0,0.0
3,1,2013-01-28,AA,N3AMAA,618,SFO,13930,1393002,ORD,19.0,4.0,0.0
4,1,2013-01-07,AA,N456AA,153,MCO,13930,1393002,ORD,806.0,-4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2106102,7,2019-11-17,OO,N930SW,5829,FAR,13930,1393007,ORD,2017.0,-8.0,20.0
2106103,7,2019-11-17,OO,N121SY,5852,TYS,13930,1393007,ORD,714.0,21.0,7.0
2106104,7,2019-11-17,OO,N203SY,5854,LGA,13930,1393007,ORD,1816.0,1.0,18.0
2106105,7,2019-11-17,OO,N145SY,5887,PWM,13930,1393007,ORD,803.0,-39.0,8.0


## Wrangle Seat Data - FAA

Code to create start draft of plane_names.csv:  
```
plane_details.drop(columns=['tail_num']).drop_duplicates().to_csv('../data/plane_names.csv')
````

In [293]:
# Fix duplicate manufacturer names
names = ['AIRBUS', 'CANADAIR', 'DASSAULT', 'DIAMOND AIRCRAFT', 'EMBRAER', 'GATES LEARJET', 'MCDONNELL DOUGLAS', 'PIPER', 'RAYTHEON', 'ROBINSON HELICOPTER']
for name in names:
    plane_details['manufacturer'].replace(regex=True,inplace=True,to_replace=rf'.*{name}.*',value=rf'{name}')
# Add FAA descriptions
plane_details = pd.merge(plane_details, plane_names, how='left', on=['manufacturer', 'model'])

In [294]:
plane_details

,tail_num,manufacturer,model,description
0,N582AA,MCDONNELL DOUGLAS,DC-9-82(MD-82),McDonnell Douglas DC9 Super 80/MD81/82/83/88
1,N439AA,MCDONNELL DOUGLAS,DC-9-83(MD-83),McDonnell Douglas DC9 Super 80/MD81/82/83/88
2,N553AA,MCDONNELL DOUGLAS,DC-9-82(MD-82),McDonnell Douglas DC9 Super 80/MD81/82/83/88
3,N558AA,MCDONNELL DOUGLAS,DC-9-82(MD-82),McDonnell Douglas DC9 Super 80/MD81/82/83/88
4,N536AA,AMERICAN AIRCRAFT INC,FALCON XP,NaN
...,...,...,...,...
4507,N324DX,AIRBUS,A321-211,Airbus Industrie A321
4508,N384DN,AIRBUS,A321-211,Airbus Industrie A321
4509,N872DN,BOEING,737-900ER,Boeing 737-900ER
4510,N406AN,AIRBUS,A321-253NX,Airbus Industrie A321-200n


In [134]:
ORD_seats_types = pd.merge(ORD_seats, aircraft_types, how='left', left_on='AIRCRAFT_TYPE', right_on='Code')

Code to create L_AIRCRAFT_TYPE_with_seats.csv:  
```
aircraft_types_seats = combined_ord_seats_types[['DEPARTURES_PERFORMED', 'SEATS', 'Description']].groupby('Description').sum()  
aircraft_types_seats['plane_seats'] = aircraft_types_seats.apply(lambda row: row.SEATS/row.DEPARTURES_PERFORMED, axis = 1)
aircraft_types_seats[['plane_seats']].to_csv("../data/L_AIRCRAFT_TYPE_with_seats.csv")
```

In [295]:
seat_counts_FAA = ORD_seats_types.query('DEPARTURES_PERFORMED > 0.0').groupby(['UNIQUE_CARRIER','Description']).sum().reset_index()
seat_counts_FAA['seats'] = seat_counts_FAA.apply(lambda row: row.SEATS/row.DEPARTURES_PERFORMED, axis = 1)
seat_counts_FAA = seat_counts_FAA[['UNIQUE_CARRIER', 'Description', 'seats']]

In [296]:
seat_counts_FAA

,UNIQUE_CARRIER,Description,seats
0,04Q,Cessna Citation X Model 650/550B/550XL,8.000000
1,09Q,Boeing 737-300,142.100000
2,09Q,Boeing 737-400,127.513924
3,09Q,Boeing 737-800,165.600000
4,09Q,Boeing 767-200/ER/EM,195.100000
...,...,...,...
252,YV,Canadair RJ-700,66.663157
253,YV,Embraer ERJ-175,76.000000
254,YX,Embraer ERJ-175,75.946044
255,YX,Embraer-Emb-170,69.935496


## Wrangle Seat Data - Wikipedia

In [297]:
seat_data = seat_data.drop(columns=['Unnamed: 0'])

In [326]:
seat_data.head(5)

,airline,aircraft,in_service,pass_count
0,9E,Bombardier CRJ-200,42,50.0
1,9E,Bombardier CRJ-700,14,69.0
2,9E,Bombardier CRJ-900,3,70.0
3,9E,Bombardier CRJ-900,116,76.0
4,AA,Airbus A319-100,133,128.0


In [333]:
seat_data.groupby('airline').sum()

,in_service,pass_count
airline,,
9E,175,265.0
AA,1924,3949.0
AS,234,1247.0
B6,264,971.0
DL,1244,5135.0
EV,118,170.0
F9,95,596.0
MQ,173,170.0
NK,153,737.0


## Merge data together

In [313]:
ORD_OTP_with_tails = pd.merge(ORD_OTP, plane_details, how='left', left_on=['TAIL_NUM'], right_on=['tail_num'])

In [317]:
seat_counts_FAA.head(1)

,UNIQUE_CARRIER,Description,seats
0,04Q,Cessna Citation X Model 650/550B/550XL,8.0


In [318]:
ORD_OTP_with_tails_seats = pd.merge(ORD_OTP_with_tails, seat_counts_FAA, how='left', left_on=['OP_UNIQUE_CARRIER', 'description'], right_on=['UNIQUE_CARRIER', 'Description'])

In [323]:
ORD_OTP_with_tails_seats

,Unnamed: 0,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,ARR_TIME,ARR_DELAY,ARR_HOUR,tail_num,manufacturer,model,description,UNIQUE_CARRIER,Description,seats
0,19,1,2013-01-07,AA,N3DEAA,618,SFO,13930,1393002,ORD,7.0,-8.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20,1,2013-01-14,AA,N3DDAA,618,SFO,13930,1393002,ORD,10.0,-5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21,1,2013-01-21,AA,N3DJAA,618,SFO,13930,1393002,ORD,7.0,-8.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,1,2013-01-28,AA,N3AMAA,618,SFO,13930,1393002,ORD,19.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90,1,2013-01-07,AA,N456AA,153,MCO,13930,1393002,ORD,806.0,-4.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106102,602391,7,2019-11-17,OO,N930SW,5829,FAR,13930,1393007,ORD,2017.0,-8.0,20.0,N930SW,BOMBARDIER INC,CL-600-2B19,Canadair RJ-200ER /RJ-440,OO,Canadair RJ-200ER /RJ-440,50.000000
2106103,602412,7,2019-11-17,OO,N121SY,5852,TYS,13930,1393007,ORD,714.0,21.0,7.0,N121SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170,NaN,NaN,NaN
2106104,602414,7,2019-11-17,OO,N203SY,5854,LGA,13930,1393007,ORD,1816.0,1.0,18.0,N203SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170,NaN,NaN,NaN
2106105,602441,7,2019-11-17,OO,N145SY,5887,PWM,13930,1393007,ORD,803.0,-39.0,8.0,N145SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170,NaN,NaN,NaN


In [77]:
ORD_outbound_weather = pd.merge(ORD_outbound, ORD_weather_hourly, how='left', on=['year', 'month', 'day', 'hour'])

In [78]:
ORD_outbound_weather

,Unnamed: 0,pickup_community_area,year,month,day,hour,rides,tmpf,dwpf,relh,sknt,p01i,vsby,skyl1,skyl2,skyl3,feel
0,0,76,2013,1,1,0,22,24.98,17.96,74.290000,9.000000,0.0000,9.000000,1800.000000,13000.000000,19000.0,14.780000
1,1,76,2013,1,1,1,9,24.89,17.78,73.995000,9.000000,0.0000,8.000000,1800.000000,14000.000000,NaN,14.670000
2,2,76,2013,1,1,2,11,21.20,14.00,73.320000,10.000000,0.0000,9.000000,1800.000000,11000.000000,15000.0,9.420000
3,3,76,2013,1,1,3,3,21.14,12.14,67.723333,10.666667,0.0000,9.666667,5400.000000,12000.000000,15000.0,8.976667
4,4,76,2013,1,1,4,5,19.94,10.94,67.580000,10.000000,0.0000,10.000000,2100.000000,9500.000000,NaN,7.840000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61977,61977,76,2020,1,31,20,257,37.00,30.00,75.560000,5.076923,0.0001,7.615385,1730.384615,NaN,NaN,31.550000
61978,61978,76,2020,1,31,21,243,37.00,30.00,75.560000,5.153846,0.0000,7.615385,1745.384615,2500.000000,NaN,31.550000
61979,61979,76,2020,1,31,22,154,37.00,30.00,75.560000,4.714286,0.0000,7.000000,2064.214286,2723.076923,NaN,32.330000
61980,61980,76,2020,1,31,23,77,36.00,28.90,75.150000,5.076923,0.0000,8.615385,1976.230769,NaN,NaN,30.350000
